
<a href="https://colab.research.google.com/github/NeuroDesk/example-notebooks/blob/main/books/workflows/MRIQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Google Colab"/>   </a>

# MRIQC

Author: Monika Doerig

Citation:

__MRIQC__:

- Esteban O, Birman D, Schaer M, Koyejo OO, Poldrack RA, Gorgolewski KJ; MRIQC: Advancing the Automatic Prediction of Image Quality in MRI from Unseen Sites; PLOS ONE 12(9):e0184661; doi:[10.1371/journal.pone.0184661](https://doi.org/10.1371/journal.pone.0184661)

- The documentation of this project is found [here](https://mriqc.readthedocs.io/).
  

__Data from OpenNeuro:__ Flanker Dataset

- Kelly AMC and Uddin LQ and Biswal BB and Castellanos FX and Milham MP (2018). Flanker task (event-related). [OpenNeuro Dataset ds000102](https://openneuro.org/datasets/ds000102/versions/00001/). [Dataset] doi: null
- Kelly AM, Uddin LQ, Biswal BB, Castellanos FX, Milham MP. Competition between functional brain networks mediates behavioral variability. Neuroimage. 2008 Jan 1;39(1):527-37. doi: [10.1016/j.neuroimage.2007.08.008](https://doi.org/10.1016/j.neuroimage.2007.08.008). Epub 2007 Aug 23. PMID: 17919929.
- Mennes, M., Kelly, C., Zuo, X.N., Di Martino, A., Biswal, B.B., Castellanos, F.X., Milham, M.P. (2010). Inter-individual differences in resting-state functional connectivity predict task-induced BOLD activity. Neuroimage, 50(4):1690-701. doi: [10.1016/j.neuroimage.2010.01.002](https://doi.org/10.1016/j.neuroimage.2010.01.002). Epub 2010 Jan 15. Erratum in: Neuroimage. 2011 Mar 1;55(1):434
- Mennes, M., Zuo, X.N., Kelly, C., Di Martino, A., Zang, Y.F., Biswal, B., Castellanos, F.X., Milham, M.P. (2011). Linking inter-individual differences in neural activation and behavior to intrinsic brain dynamics. Neuroimage, 54(4):2950-9. doi: [10.1016/j.neuroimage.2010.10.046](https://doi.org/10.1016/j.neuroimage.2010.10.046)

## Setup Neurodesk

In [1]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [2]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: GenuineIntel
model name	: Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz


## Load MRIQC

In [3]:
import lmod
await lmod.load('mriqc/22.0.6')
await lmod.list()

['afni/21.2.00', 'mriqc/22.0.6']

In [4]:
!mriqc -help

You are using MRIQC v22.0.6, and a newer version is available: 24.0.2.
usage: mriqc [-h] [--version] [-v] [--species {human,rat}]
             [--participant-label PARTICIPANT_LABEL [PARTICIPANT_LABEL ...]]
             [--session-id [SESSION_ID ...]] [--run-id [RUN_ID ...]]
             [--task-id [TASK_ID ...]] [-m [MODALITIES ...]] [--dsname DSNAME]
             [--bids-database-dir PATH] [--nprocs NPROCS]
             [--omp-nthreads OMP_NTHREADS] [--mem MEMORY_GB] [--testing] [-f]
             [--pdb] [-w WORK_DIR] [--verbose-reports] [--write-graph]
             [--dry-run] [--resource-monitor] [--use-plugin USE_PLUGIN]
             [--no-sub] [--email EMAIL] [--webapi-url WEBAPI_URL]
             [--webapi-port WEBAPI_PORT] [--upload-strict] [--ants-float]
             [--ants-settings ANTS_SETTINGS] [--ica] [--fft-spikes-detector]
             [--fd_thres FD_THRES] [--deoblique] [--despike]
             [--start-idx START_IDX] [--stop-idx STOP_IDX]
             bids_dir output_

## Data preparation

In [5]:
!datalad install https://github.com/OpenNeuroDatasets/ds000102.git
!cd ds000102 && datalad get sub-01 sub-02 sub-03

Cloning:   0%|                             | 0.00/2.00 [00:00<?, ? candidates/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Counting:   0%|                               | 0.00/27.0 [00:00<?, ? Objects/s]
                                                                                
Compressing:   0%|                            | 0.00/23.0 [00:00<?, ? Objects/s]
                                                                                
Receiving:   0%|                             | 0.00/2.15k [00:00<?, ? Objects/s]
                                                                                
Resolving:   0%|                                | 0.00/537 [00:00<?, ? Deltas/s]
[INFO   ] scanning for unlocked files (this may take some time)                 
[INFO   ] Remote origin not usable by git-annex; setting annex-ignore 
[INFO   ] access to 1 dataset sibling s3-PRIVATE not auto-enabled, enable with:
| 		datalad siblings -d "/

## Running MRIQC

The following section outlines key MRIQC command-line options used in this analysis. These include required positional arguments for specifying input and output directories, as well as optional flags for performance optimization and report generation. For a complete list of available options, refer to the [official documentation](https://mriqc.readthedocs.io/en/latest/usage.html#positional-arguments).


#### Positional Arguments
- `bids_dir`: The root folder of a BIDS valid dataset (sub-XXXXX folders should be found at the top level in this folder).

- `output_dir`:  The directory where the output files should be stored. If you are running group level analysis this folder should be prepopulated with the results of the participant level analysis.

- `analysis_level`:  Possible choices: participant, group

Level of the analysis that will be performed. Multiple participant level analyses can be run independently (in parallel) using the same output_dir.

- `-v, --verbose`: Increases log verbosity for each occurrence, debug level is -vvv.


#### Options to handle performance
- `--nprocs, --n_procs, --n_cpus, -n-cpus`
Maximum number of simultaneously running parallel processes executed by MRIQC (e.g., several instances of ANTs’ registration). However, when --nprocs is greater or equal to the --omp-nthreads option, it also sets the maximum number of threads that simultaneously running processes may aggregate (meaning, with --nprocs 16 --omp-nthreads 8 a maximum of two 8-CPU-threaded processes will be running at a given time). Under this mode of operation, --nprocs sets the maximum number of processors that can be assigned work within an MRIQC job, which includes all the processors used by currently running single- and multi-threaded processes. If None, the number of CPUs available will be automatically assigned (which may not be what you want in, e.g., shared systems like a HPC cluster.

- `--mem, --mem_gb, --mem-gb`: Upper bound memory limit for MRIQC processes.

#### Instrumental options
- `-w, --work-dir`:
Path where intermediate results should be stored.

- `--verbose-reports`: Generate verbose HTML reports with additional details
- `--no-sub`: Turn off submission of anonymized quality metrics to MRIQC’s metrics repository.


<div style="background-color: #ffe5cc; padding: 10px; border-radius: 5px;">
    <div style="background-color: #ff9900; padding: 5px; border-radius: 5px; font-weight: bold;">
        <span style="font-size: 15px; color: #ff9900; background-color: white; border-radius: 50%; width: 20px; height: 20px; display: inline-block; text-align: center; line-height: 20px;">
            !
        </span> <span style="color: white;">Warning:</span>
    </div>
    <p style="margin: 10px 0;">
<code>--no-sub</code> and <code>--notrack</code> may be necessary in environments with restricted internet access, where submission attempts may trigger runtime errors.
    </p>
</div>

### Participant-level Analysis

In [6]:
%%bash
# Set the number of threads for ITK (Image Processing Toolkit) to use
export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=6
# Set the directory for matplotlib configuration to avoid conflicts
export MPLCONFIGDIR=~/matplotlib-mpldir

# Run MRIQC
mriqc ds000102 \
        MRIQC \
        participant \
        --participant-label sub-01 sub-02 sub-03 \
        --no-sub \
        --work-dir MRIQC_workdir \
        --nprocs 6 \
        --mem-gb 10000 \
        --verbose-reports \
        -v

You are using MRIQC v22.0.6, and a newer version is available: 24.0.2.


250506-06:07:42,500 cli IMPORTANT:
	 
    Running MRIQC version 22.0.6:
      * BIDS dataset path: /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102.
      * Output folder: /home/jovyan/Git_repositories/example-notebooks/books/workflows/MRIQC.
      * Analysis levels: ['participant'].

250506-06:07:42,739 nipype.workflow INFO:
	 Building functional MRIQC workflow for files: /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz, /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz, /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102/sub-02/func/sub-02_task-flanker_run-1_bold.nii.gz, /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102/sub-02/func/sub-02_task-flanker_run-2_bold.nii.gz, /home/jovyan/Git_repositories/example-notebooks/books/workflows/ds000102/sub-03/func/sub-03_task-flanker

### Group-level Analysis

In [7]:
%%bash
mriqc ds000102 \
        MRIQC \
        group \
        --no-sub \
        --work-dir MRIQC_workdir \
        --nprocs 6 \
        --mem-gb 10000 \
        --verbose-reports \
        -v                           

You are using MRIQC v22.0.6, and a newer version is available: 24.0.2.


## MRIQC Visual Reports
MRIQC automatically generates individual and group-level reports to support the quality assessment of MRI data.

Individual reports include mosaic views of anatomical or functional images along various cutting planes, along with overlays like segmentation contours or motion estimates. These visuals help quickly screen for artifacts or other issues. They are especially useful for reviewing images that may have been flagged as low-quality by the automated classifier.

Group reports compile quality metrics across all subjects and display interactive scatter plots for each image quality metric (IQM). These plots make it easy to spot outliers—clicking on any point in the scatter plot opens the corresponding individual report for that subject.

These embedded reports (shown below via IFrames) offer an intuitive way to explore quality metrics and support decisions about excluding or further examining specific runs or subjects.

### Anatomical Report

In [8]:
from IPython.display import IFrame

IFrame(src='./MRIQC/sub-01_T1w.html', width='100%', height='800px')

### Functional Report

In [9]:
IFrame(src='./MRIQC/sub-02_task-flanker_run-1_bold.html', width='100%', height='800px')

### Group functional report

In [10]:
IFrame(src='./MRIQC/group_bold.html', width='100%', height='800px')